# Computing distribution for 1-dimensional moduli space

In [ ]:
import plotly.graph_objects as go

from singdistance import *

logger.setLevel(logging.DEBUG)

In [ ]:
m1 = ModuliSpace(2.7, (3.5, 3))

In [ ]:
%%time
m1.ode_steps = 10000
m1.calc_area_distribution()

In [ ]:
dist = m1.a_distribution
x = dist.steps[:-1]
y = dist.values
r = [m1.rhs_function(x) for x in x]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
# fig.add_trace(go.Scatter(x=x, y=r))
fig['layout'].update(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

print(f'Estimated volume: {((x[1:] - x[:-1]) * y[:-1]).sum()}, Actual volume: {m1.thurston_volume}')

In [ ]:
alpha1, alpha2 = [m1.alpha[0]], [m1.alpha[1]]

limits = max(0, m1.phi1 - alpha2[0]) + 1e-5, min(alpha1[0], m1.phi1) - 1e-5

m = np.exp(-1j * alpha1[0] / 2) / np.sin(alpha1[0] / 2) + \
    np.exp(1j * (alpha2[0] / 2 - m1.phi1)) / np.sin(alpha2[0] / 2)
m = np.abs(m) / 2
c = - 1 / np.tan(alpha1[0] / 2) / 2 - 1 / np.tan(alpha2[0] / 2) / 2

In [ ]:
def f_an(a):
    # https://tinyurl.com/ansolution
    q = m1.q

    w1 = ((1 - c * q)**2 - m**2 * q**2) ** .5
    w2 = (a - c) * (1 - c * q) - m**2 * q
    w3 = (m**2 - (a - c)**2) ** .5

    res = np.arctan(w2/w3/w1) / w1 / q / a**2
    res += np.arctan((c - a) / w3) / q / a**2
    return res

In [ ]:
x = dist.steps[:-1]
y = dist.values

k1 = y[3000] / x[3000] ** -2  # choose point in the 4th part

y_an = np.array([f_an(x) + k1 * x ** -2 for x in x])
y_an = np.minimum(np.maximum(y_an, -5), 5)

k2 = (y[300] - f_an(x[300]) / 2) / x[300] ** -2  # choose point in second part

y_an2 = np.array([f_an(x) / 2 + k2 * x ** -2 for x in x])
y_an2 = np.minimum(np.maximum(y_an2, -5), 5)

y_an3 = k1 * x ** -2
y_an3 = np.minimum(np.maximum(y_an3, -5), 5)

y_an4 = 0 * x

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y_an))
fig.add_trace(go.Scatter(x=x, y=y_an2))
fig.add_trace(go.Scatter(x=x, y=y_an3))
fig.add_trace(go.Scatter(x=x, y=y_an4))
fig.add_trace(go.Scatter(x=x, y=y))
fig['layout'].update(margin=dict(l=0, r=0, b=0, t=20))
fig.show()